<a href="https://colab.research.google.com/github/ramneekkaur/Deep_Learning/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn
import numpy as np

In [0]:
text=['hey how are you','i am fine','good i m great','we can meet','great sure']
chars=set(''.join(text))

int2char=dict(enumerate(chars))

char2int={char:ind for ind, char in int2char.items()}

In [0]:
print(int2char)

{0: ' ', 1: 'g', 2: 'u', 3: 'e', 4: 'w', 5: 'r', 6: 'm', 7: 'c', 8: 's', 9: 'a', 10: 'd', 11: 'n', 12: 't', 13: 'y', 14: 'f', 15: 'h', 16: 'o', 17: 'i'}


In [0]:
print(char2int)

{' ': 0, 'g': 1, 'u': 2, 'e': 3, 'w': 4, 'r': 5, 'm': 6, 'c': 7, 's': 8, 'a': 9, 'd': 10, 'n': 11, 't': 12, 'y': 13, 'f': 14, 'h': 15, 'o': 16, 'i': 17}


In [0]:
maxlen=len(max(text,key=len))

for i in range(len(text)):
  while len(text[i])<maxlen:
    text[i] += ' '

In [0]:
input_seq=[]
target_seq=[]

for i in range(len(text)):
  
  input_seq.append(text[i][:-1])
  
  target_seq.append(text[i][1:])
  print("Input Sequence:{}\n Target sequence: {}".format(input_seq[i],target_seq[i]))

Input Sequence:hey how are yo
 Target sequence: ey how are you
Input Sequence:i am fine     
 Target sequence:  am fine      
Input Sequence:good i m great
 Target sequence: ood i m great 
Input Sequence:we can meet   
 Target sequence: e can meet    
Input Sequence:great sure    
 Target sequence: reat sure     


In [0]:
for i in range(len(text)):
  input_seq[i] = [char2int[character] for character in input_seq[i]]
  target_seq[i]= [char2int[character] for character in target_seq[i]]

In [0]:
dict_size= len(char2int)
seq_len=maxlen-1
batch_size=len(text)

def one_hot_encode(sequence,dict_size, seq_len, batch_size):
  features=np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
  
  
  for i in range(batch_size):
    for u in range(seq_len):
      features[i, u, sequence[i][u]]=1
  return features

In [0]:
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)

In [0]:
input_seq=torch.from_numpy(input_seq)
target_seq=torch.Tensor(target_seq)

In [0]:
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [0]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [0]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

print (model)
# Define hyperparameters
n_epochs = 100
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Model(
  (rnn): RNN(18, 12, batch_first=True)
  (fc): Linear(in_features=12, out_features=18, bias=True)
)


In [0]:
# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    input_seq.to(device)
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 2.3103
Epoch: 20/100............. Loss: 2.1319
Epoch: 30/100............. Loss: 1.9160
Epoch: 40/100............. Loss: 1.6363
Epoch: 50/100............. Loss: 1.3338
Epoch: 60/100............. Loss: 1.0600
Epoch: 70/100............. Loss: 0.8028
Epoch: 80/100............. Loss: 0.5856
Epoch: 90/100............. Loss: 0.4305
Epoch: 100/100............. Loss: 0.3263


In [0]:
# This function takes in the model and character as arguments and returns the next character prediction and hidden state
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character.to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [0]:
# This function takes the desired output length and input characters as arguments, returning the produced sentence
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [0]:
sample(model,15, 'hey')

'hey how are you'